# Traitement des données

In [ ]:
import numpy as np
import pylab as pl
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

X_train = pd.read_csv('X_train_HI5.csv')
X_test = pd.read_csv('X_test_HI5.csv')

y_train = X_train['piezo_groundwater_level_category']
X_train = X_train.drop(['piezo_groundwater_level_category'], axis=1) 

## I. Cohérence des types

### 1.1 Piezo variables

In [ ]:
# Filtrer les colonnes commençant par "piezo"
piezo_columns = [col for col in X_train.columns if col.startswith('piezo')]

# Vérifier les types des colonnes sélectionnées
piezo_types = X_train[piezo_columns].dtypes

# Afficher les résultats
piezo_types


**Dates (actuellement object) :**
Les colonnes piezo_station_update_date et piezo_measurement_date sont des dates, mais elles sont au format object.
Action : Convertir ces colonnes en type datetime.

In [ ]:
X_train['piezo_station_update_date'] = X_train['piezo_station_update_date'].str.replace(r'CEST', '', regex=True).str.strip()
X_train['piezo_measurement_date'] = X_train['piezo_measurement_date'].str.replace(r'CEST', '', regex=True).str.strip()
X_train['piezo_station_update_date'] = pd.to_datetime(X_train['piezo_station_update_date'], errors='coerce')
X_train['piezo_measurement_date'] = pd.to_datetime(X_train['piezo_measurement_date'], errors='coerce')


**Codes et identifiants (actuellement object) :** 

Colonnes concernées :  

piezo_station_department_code, piezo_station_bdlisa_codes, piezo_station_bss_code, piezo_station_bss_id, piezo_bss_code.  

Ces colonnes contiennent des codes ou identifiants. Si elles ne sont pas utilisées comme valeurs numériques, elles peuvent rester au format object. Sinon, vérifier s'il est pertinent de les convertir en category pour économiser de la mémoire.  

Action : Convertir en category si elles ne nécessitent pas d'opérations numériques.

In [ ]:
cols_to_category = [
    'piezo_station_department_code', 'piezo_station_bdlisa_codes', 
    'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code'
]
X_train[cols_to_category] = X_train[cols_to_category].astype('category')


**Catégories qualitatives (actuellement object) :**  

Colonnes concernées :  

piezo_station_department_name, piezo_station_commune_name, piezo_obtention_mode, piezo_status, piezo_qualification, piezo_continuity_name, piezo_producer_name, piezo_measure_nature_code, piezo_measure_nature_name.  

Ces colonnes semblent représenter des catégories qualitatives (ex. : nom de département, nature de la mesure).  

Action : Convertir en type category si elles sont destinées à être utilisées pour le regroupement, l'analyse ou la modélisation.

In [ ]:
qualitative_cols = [
    'piezo_station_department_name', 'piezo_station_commune_name', 
    'piezo_obtention_mode', 'piezo_status', 'piezo_qualification', 
    'piezo_continuity_name', 'piezo_producer_name', 
    'piezo_measure_nature_code', 'piezo_measure_nature_name'
]
X_train[qualitative_cols] = X_train[qualitative_cols].astype('category')


**Colonnes numériques (déjà float64)**

Colonnes concernées :   

piezo_station_investigation_depth, piezo_station_altitude, piezo_station_longitude, piezo_station_latitude, piezo_continuity_code, piezo_producer_code.   

Ces colonnes semblent correctement typées. Vérifie cependant la présence d'erreurs ou de valeurs aberrantes (NaN, valeurs extrêmes).   

Action : Analyser les valeurs aberrantes et les NaN pour décider d'un traitement adapté.

In [ ]:
print(X_train[['piezo_station_investigation_depth', 'piezo_station_altitude']].describe())


### 1.2 Meteo variables

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.4f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [ ]:
# Filtrer les colonnes commençant par "meteo_"
meteo_columns = [col for col in X_train.columns if col.startswith('meteo_')]

# Vérifier les types des colonnes sélectionnées
meteo_types = X_train[meteo_columns].dtypes

# Calculer les statistiques descriptives pour les colonnes quantitatives
print_full(meteo_types)



**Dates (meteo_date)**

Actuellement au format object, ce qui limite les manipulations.  

Action : Convertir en type datetime pour permettre les analyses temporelles.

In [ ]:
X_train['meteo_date'] = pd.to_datetime(X_train['meteo_date'], errors='coerce')


**Identifiants et noms (meteo_id, meteo_name)**

meteo_id (station ID) : Correctement typé en int64.  

meteo_name (nom de la station) : Peut être converti en type category si utilisé pour le regroupement ou l'analyse.  

Action : Convertir meteo_name en category.

In [ ]:
X_train['meteo_name'] = X_train['meteo_name'].astype('category')


**3. Variables booléennes encodées en float64**

Colonnes : meteo_if_snow, meteo_if_fog, meteo_if_thunderstorm, etc. 

Ces colonnes semblent représenter des indicateurs booléens (présence/absence). 

Action : Convertir en type bool.

In [ ]:
bool_columns = [
    'meteo_if_snow', 'meteo_if_fog', 'meteo_if_thunderstorm', 'meteo_if_sleet', 'meteo_if_hail',
    'meteo_if_dew', 'meteo_if_black_ice', 'meteo_if_snow_ground', 'meteo_if_frost', 
    'meteo_if_smoke', 'meteo_if_mist', 'meteo_if_lightning'
]
X_train[bool_columns] = X_train[bool_columns].astype('bool')


**Variables quantitatives (float64)**

Les variables comme meteo_temperature_min, meteo_radiation, etc., sont correctement typées. 

Action : Vérifier les valeurs aberrantes et remplacer les NaN par des stratégies spécifiques (moyenne, interpolation, etc.).

In [ ]:
# Identifier les colonnes quantitatives
quantitative_vars = X_train[meteo_columns].select_dtypes(include=['float64', 'int64']).columns
print(quantitative_vars)

# Calculer les statistiques descriptives pour les colonnes quantitatives
print_full(X_train[quantitative_vars].describe())


### 1.3 Hydro variables

In [ ]:
# Filtrer les colonnes commençant par "hydro_"
hydro_columns = [col for col in X_train.columns if col.startswith('hydro_')]

# Vérifier les types des colonnes sélectionnées
hydro_types = X_train[hydro_columns].dtypes

# Calculer les statistiques descriptives pour les colonnes quantitatives
hydro_quantitative = X_train[hydro_columns].select_dtypes(include=['float64', 'int64']).describe()

# Afficher les résultats des types et des statistiques
print_full(hydro_types)


**Colonnes qualitatives (object)**




In [ ]:
X_train['hydro_station_code'] = X_train['hydro_station_code'].astype('category')
qualitative_columns = ['hydro_status_label', 'hydro_method_label', 'hydro_qualification_label']
X_train[qualitative_columns] = X_train[qualitative_columns].astype('category')


In [ ]:
# Identifier les valeurs uniques et leur fréquence dans "hydro_hydro_quantity_elab"
hydro_quantity_values = X_train['hydro_hydro_quantity_elab'].value_counts(dropna=False)

# Vérifier les valeurs manquantes
hydro_quantity_nan = X_train['hydro_hydro_quantity_elab'].isna().sum()

# Afficher les résultats
print("Fréquence des valeurs uniques :")
print(hydro_quantity_values)

print("\nNombre de valeurs manquantes :")
print(hydro_quantity_nan)


In [ ]:
X_train['hydro_hydro_quantity_elab'] = X_train['hydro_hydro_quantity_elab'].astype('category')


In [ ]:
# Analyse globale pour "hydro_observation_result_elab"
global_stats = X_train['hydro_observation_result_elab'].describe()
global_nan = X_train['hydro_observation_result_elab'].isna().sum()

print("Analyse globale pour 'hydro_observation_result_elab':")
print(global_stats)
print(f"\nNombre de valeurs manquantes : {global_nan}")

# Analyse par type de "hydro_hydro_quantity_elab" (QmJ et QmM)
grouped_stats = X_train.groupby('hydro_hydro_quantity_elab')['hydro_observation_result_elab'].describe()
grouped_nan = X_train.groupby('hydro_hydro_quantity_elab')['hydro_observation_result_elab'].apply(lambda x: x.isna().sum())

print("\nAnalyse par type de 'hydro_hydro_quantity_elab':")
print(grouped_stats)
print("\nNombre de valeurs manquantes par type de 'hydro_hydro_quantity_elab':")
print(grouped_nan)


--> hydro_hydro_quantity_elab ne correspond qu'au mode de calcul et n'affecte pas l'échelle de donnée entre QmJ et QmM 

**Dates (object)**

Description : Date de l’observation élaborée.

Recommandation :
Convertir en datetime pour permettre une analyse temporelle.

In [ ]:
X_train['hydro_observation_date_elab'] = pd.to_datetime(X_train['hydro_observation_date_elab'], errors='coerce')

**Colonnes numériques**

In [ ]:
hydro_quantitative.describe()

### 1.4 Prelev variables

In [ ]:
# Filtrer les colonnes commençant par "prelev_"
prelev_columns = [col for col in X_train.columns if col.startswith('prelev_')]

# Vérifier les types des colonnes "prelev_"
prelev_types = X_train[prelev_columns].dtypes

# Afficher les résultats
prelev_types


**Colonnes qualitatives (object)**

In [ ]:
# Identifier les colonnes descriptives (object) commençant par "prelev_"
prelev_object_cols = X_train[[col for col in prelev_columns if X_train[col].dtype == 'object']]

# Convertir les colonnes en type "category"
X_train[prelev_object_cols.columns] = X_train[prelev_object_cols.columns].astype('category')


**Variables quantitatives (float64)**


In [ ]:
# Calculer les statistiques descriptives pour les colonnes quantitatives "prelev_"
prelev_quantitative = X_train[prelev_columns].select_dtypes(include=['float64', 'int64']).describe()

prelev_quantitative

In [ ]:
# Identifier les colonnes contenant "commune_code_insee" dans "prelev_"
insee_columns = [col for col in prelev_columns if 'commune_code_insee' in col]

# Convertir les colonnes INSEE en type "category"
X_train[insee_columns] = X_train[insee_columns].astype('category')

# Vérifier le résultat de la conversion
insee_converted_types = X_train[insee_columns].dtypes
insee_converted_types


### 1.5 INSEE variables

In [ ]:
# Identifier les colonnes commençant par "insee"
insee_columns = [col for col in X_train.columns if col.startswith('insee')]

# Vérifier les types des colonnes "insee"
insee_types = X_train[insee_columns].dtypes

# Résultat des types avant et après conversion
print_full(insee_types)


In [ ]:
# Colonnes "insee" à convertir en type float
insee_columns_to_convert = ['insee_%_agri', 'insee_med_living_level', 'insee_%_ind', 'insee_%_const']

# Conversion en type float
X_train[insee_columns_to_convert] = X_train[insee_columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Vérifier les types après conversion
print("Types des colonnes après conversion en float :")
print(X_train[insee_columns_to_convert].dtypes)


### 1.6 Distance variables

In [ ]:
# Identifier les colonnes contenant "distance_"
distance_columns = [col for col in X_train.columns if 'distance_' in col]

# Vérifier les types des colonnes "distance_"
distance_types = X_train[distance_columns].dtypes

# Calculer les statistiques descriptives pour les colonnes "distance_"
distance_stats = X_train[distance_columns].describe()

# Afficher les résultats
distance_types, distance_stats


## II. Suppression de données récurentes / inutiles

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.1 Analyse des variables : Piezo

| Variable                           | Description                                                                                     | Décision                                                                                              |
|------------------------------------|-------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| `piezo_station_department_code`    | Code du département français.                                                                  | **Conserver** : Référence géographique utile pour l'analyse et les groupements régionaux.            |
| `piezo_station_update_date`        | Date de la dernière mise à jour de la station.                                                 | **Conserver** : Fournit une référence chronologique importante.                                       |
| `piezo_station_investigation_depth`| Profondeur de la sonde.                                                                        | **Conserver** : Pertinent pour l'influence sur le niveau piézométrique.                              |
| `piezo_station_department_name`    | Nom du département.                                                                            | **Supprimer** : Doublon avec `piezo_station_department_code`.                                        |
| `piezo_station_commune_code_insee` | Code INSEE de la commune où se trouve la station.                                              | **Conserver** : Permet de lier la station à des données socio-économiques ou climatiques.            |
| `piezo_station_pe_label`           | Nom de la nappe piézométrique.                                                                 | **Conserver** : Relie les mesures au système aquifère sous-jacent.                                   |
| `piezo_station_bdlisa_codes`       | Codes des entités hydrogéologiques (BDLISA).                                                   | **Conserver** : Essentiel pour caractériser les propriétés hydrogéologiques.                         |
| `piezo_station_altitude`           | Altitude du sol au niveau de la station (NGF).                                                 | **Conserver** : Influence directe sur le niveau d'eau souterraine.                                   |
| `piezo_station_bss_code`           | Code(s) national(aux) de la station (codes anciens).                                           | **Conserver** : Référence unique utile pour compléter `bdlisa_codes`.                                |
| `piezo_station_commune_name`       | Nom de la commune.                                                                             | **Supprimer** : Doublon avec `piezo_station_commune_code_insee`.                                     |
| `piezo_station_longitude`          | Coordonnée X de la station.                                                                    | **Conserver** : Utile pour les analyses géospatiales.                                                |
| `piezo_station_latitude`           | Coordonnée Y de la station.                                                                    | **Conserver** : Utile pour les analyses géospatiales.                                                |
| `piezo_station_bss_id`             | Code(s) national(aux) de la station (codes récents).                                           | **Supprimer** : Identifiant sans impact direct sur les prédictions.                                  |



In [ ]:
# Liste des colonnes à supprimer
columns_to_drop = [
    'piezo_station_department_name',    # Doublon avec piezo_station_department_code
    'piezo_station_commune_name',       # Doublon avec piezo_station_commune_code_insee
    'piezo_station_bss_id'              # Identifiant non nécessaire pour la prédiction
]

# Suppression des colonnes dans X_train et X_test
X_train = X_train.drop(columns=columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=columns_to_drop, errors='ignore')

# Vérification
print("Colonnes supprimées :", columns_to_drop)


In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.2 Analyse des variables : Meteo

### Meteo (Weather)

| **Variable**                              | **Description**                                                                                      | **Décision**                            | **Justification**                                                                                                               |
|-------------------------------------------|------------------------------------------------------------------------------------------------------|------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------|
| `meteo_cloudiness_height`                 | Minimum height of cloud layer >300m                                                                  | **Conserver**                           | Peut influencer les précipitations ou l'évaporation, liées à la recharge de la nappe.                                            |
| `meteo_if_fog`                            | Fog occurrence                                                                                       | **Supprimer**                           | Influence marginale sur le niveau de la nappe.                                                                                   |
| `meteo_if_mist`                           | Haze occurrence                                                                                      | **Supprimer**                           | Faible lien avec la recharge ou la consommation d'eau.                                                                            |
| `meteo_frost_duration`                    | Freezing duration                                                                                    | **Conserver**                           | Impact potentiel sur les variations saisonnières et l'infiltration.                                                              |
| `meteo_wetting_duration`                  | Wetting duration                                                                                     | **Conserver**                           | Indicateur pertinent des conditions humides.                                                                                      |
| `meteo_humidity_duration_below_40%`       | Duration of humidity <=40%                                                                           | **Conserver**                           | Important pour les périodes de sécheresse impactant la recharge de la nappe.                                                     |
| `meteo_humidity_duration_above_80%`       | Duration of humidity >=80%                                                                           | **Conserver**                           | Pertinent pour identifier des périodes favorables à la recharge.                                                                 |
| `meteo_radiation_direct`                  | Direct solar radiation level                                                                         | **Supprimer**                           | Faible impact direct sur la nappe phréatique.                                                                                     |
| `meteo_wind_direction_max_inst`           | Maximum instant wind direction                                                                       | **Supprimer**                           | Peu de lien direct avec les niveaux de nappe.                                                                                     |
| `meteo_wind_direction_max_inst_2m`        | Maximum instant wind direction at 2 meters                                                          | **Supprimer**                           | Influence indirecte très faible sur la recharge.                                                                                 |
| `meteo_wind_direction_max_avg`            | Average maximum wind direction over 10 minutes                                                      | **Supprimer**                           | Non pertinent pour la nappe phréatique.                                                                                           |
| `meteo_if_lightning`                      | Lightning occurrence                                                                                 | **Supprimer**                           | Peu de relation avec le niveau de la nappe.                                                                                       |
| `meteo_evapotranspiration_grid`           | Evapotranspiration potential calculated at the closest grid point                                    | **Conserver**                           | Indicateur clé de la recharge et des pertes d’eau.                                                                                |
| `meteo_evapotranspiration_Monteith`       | Monteith evapotranspiration                                                                          | **Conserver**                           | Mesure essentielle pour estimer la recharge de la nappe.                                                                          |
| `meteo_wind_speed_avg_2m`                 | Average wind speed at 2 meters                                                                       | **Supprimer**                           | Peu de lien direct avec les niveaux de nappe.                                                                                     |
| `meteo_wind_speed_avg_10m`                | Average wind speed at 10 meters                                                                      | **Supprimer**                           | Même justification que ci-dessus.                                                                                                |
| `meteo_if_smoke`                          | Occurrence of smoke                                                                                  | **Supprimer**                           | Non pertinent pour la recharge de la nappe.                                                                                       |
| `meteo_wind_max`                          | Maximum instant wind speed                                                                           | **Supprimer**                           | Peu de lien direct avec le niveau de la nappe.                                                                                    |
| `meteo_wind_max_2m`                       | Maximum instant wind speed at 2 meters                                                              | **Supprimer**                           | Idem.                                                                                                                             |
| `meteo_wind_max_3s`                       | Maximum 3-second instant wind speed within an hour                                                   | **Supprimer**                           | Non pertinent pour la prédiction.                                                                                                 |
| `meteo_wind_avg`                          | Average maximum wind speed over 10 minutes                                                          | **Supprimer**                           | Peu lié à la recharge de la nappe.                                                                                                |
| `meteo_if_frost`                          | Occurrence of frost                                                                                  | **Conserver**                           | Influence indirecte sur l’infiltration et les niveaux saisonniers.                                                                |
| `meteo_radiation`                         | Global radiation level                                                                               | **Supprimer**                           | Non pertinent pour la nappe phréatique.                                                                                           |
| `meteo_if_hail`                           | Occurrence of hail                                                                                   | **Supprimer**                           | Impact négligeable sur la recharge.                                                                                               |
| `meteo_if_sleet`                          | Occurrence of sleet                                                                                  | **Supprimer**                           | Peu d’impact mesurable sur les niveaux de nappe.                                                                                  |
| `meteo_snow_height`                       | Snowfall height over 24 hours                                                                        | **Conserver**                           | Pertinent pour les périodes de recharge hivernale.                                                                                |
| `meteo_time_tn`, `meteo_time_tx`          | Time of minimum and maximum temperature in shelter                                                   | **Supprimer**                           | Les valeurs temporelles précises sont peu pertinentes.                                                                             |
| `meteo_time_humidity_min`, ...            | Times for humidity or wind events                                                                    | **Supprimer**                           | Idem.                                                                                                                             |
| `meteo_id`                                | Weather station ID                                                                                   | **Conserver**                           | Utilisé pour relier les données météorologiques à d'autres stations.                                                              |
| `meteo_radiation_IR`                      | Total hourly infrared radiation                                                                      | **Supprimer**                           | Non pertinent.                                                                                                                    |
| `meteo_sunshine_duration`                 | Sunshine duration                                                                                    | **Supprimer**                           | Relation marginale avec la recharge.                                                                                              |
| `meteo_cloudiness`                        | Maximum cloudiness above 4/8 of a layer below 300 m                                                  | **Supprimer**                           | Peu pertinent pour la recharge.                                                                                                   |
| `meteo_if_thunderstorm`, ...              | Boolean weather conditions (thunderstorm, black ice, etc.)                                           | **Supprimer**                           | Non pertinents pour la prédiction.                                                                                                |
| `meteo_rain_height`                       | Rain height                                                                                          | **Conserver**                           | Variable clé pour la recharge de la nappe phréatique.                                                                              |
| `meteo_pressure_avg`, `meteo_pressure_max`| Average and maximum sea-level pressure                                                               | **Supprimer**                           | Relation indirecte difficile à exploiter pour la prédiction.                                                                      |
| `meteo_humidity_avg`, ...                 | Average/minimum humidity                                                                             | **Conserver**                           | Pertinent pour évaluer les périodes de recharge ou d’assèchement.                                                                 |
| `meteo_temperature_avg`, ...              | Shelter temperature (average, min, max)                                                              | **Conserver**                           | Variables importantes pour comprendre l’évaporation et la recharge.                                                               |


**--> à décider ensemble !!!**

In [ ]:
# Liste des colonnes inutiles
columns_to_drop = [
    'meteo_if_fog', 'meteo_if_mist', 'meteo_radiation_direct', 
    'meteo_wind_direction_max_inst', 'meteo_wind_direction_max_inst_2m', 
    'meteo_wind_direction_max_avg', 'meteo_if_lightning', 
    'meteo_wind_speed_avg_2m', 'meteo_wind_speed_avg_10m', 
    'meteo_wind_max', 'meteo_wind_max_2m', 'meteo_wind_max_3s', 
    'meteo_wind_avg', 'meteo_radiation', 'meteo_if_hail', 
    'meteo_if_sleet', 'meteo_time_tn', 'meteo_time_tx', 
    'meteo_time_humidity_min', 'meteo_time_humidity_max', 
    'meteo_time_wind_max', 'meteo_time_wind_max_2m', 
    'meteo_time_wind_max_3s', 'meteo_radiation_IR', 
    'meteo_sunshine_duration', 'meteo_cloudiness', 
    'meteo_if_thunderstorm', 'meteo_pressure_avg', 
    'meteo_pressure_max'
]

# Suppression dans les datasets
X_train = X_train.drop(columns=columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=columns_to_drop, errors='ignore')

print(f"Colonnes supprimées : {len(columns_to_drop)}")

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.3 Analyse des variables : Hydro

| **Variable**                     | **Description**                                                                                   | **Décision**             | **Justification**                                                                                                              |
|----------------------------------|---------------------------------------------------------------------------------------------------|---------------------------|------------------------------------------------------------------------------------------------------------------------------|
| `hydro_station_code`             | Hydrometric station code                                                                          | **Supprimer**            | Les identifiants spécifiques aux stations hydrologiques ne sont pas nécessaires pour la prédiction des niveaux piezométriques. |
| `hydro_observation_date_elab`    | Hydrometric observation date                                                                      | **Conserver**            | Permet de synchroniser les données hydrologiques avec d'autres variables temporelles.                                         |
| `hydro_observation_result_elab`  | Value of the hydrometric measurement (flow rate)                                                  | **Conserver**            | Variable quantitative clé pour estimer les apports hydriques contribuant à la recharge de la nappe phréatique.               |
| `hydro_status_code`              | Code of the data's status                                                                         | **Supprimer**            | La qualité des données est déjà reflétée par d'autres indicateurs (ex. qualification).                                        |
| `hydro_status_label`             | Label/Name of the data's status                                                                   | **Supprimer**            | Redondant avec `hydro_status_code`.                                                                                          |
| `hydro_method_code`              | Code of the method for obtaining the hydro observation result                                     | **Supprimer**            | Peu pertinent pour l'analyse, les valeurs hydrométriques restent valides quelle que soit leur méthode d'obtention.           |
| `hydro_method_label`             | Method for obtaining the hydro observation result                                                | **Supprimer**            | Redondant avec `hydro_method_code`.                                                                                          |
| `hydro_qualification_code`       | Code of the quality of the measurement                                                           | **Supprimer**            | Peu utile si les données sont déjà filtrées ou nettoyées en amont.                                                           |
| `hydro_qualification_label`      | Quality of the measurement produced                                                              | **Supprimer**            | Redondant avec `hydro_qualification_code`.                                                                                   |
| `hydro_longitude`                | Longitude of the hydrometric station                                                             | **Supprimer**            | Les coordonnées géographiques des stations hydrométriques ne sont pas nécessaires, la position des stations piezo est suffisante. |
| `hydro_latitude`                 | Latitude of the hydrometric station                                                              | **Supprimer**            | Idem.                                                                                                                        |
| `hydro_hydro_quantity_elab`      | Type of hydrometric quantity developed (e.g., daily/monthly mean discharge)                      | **Conserver**            | Utile pour différencier les séries temporelles journalières et mensuelles et ajuster les modèles en conséquence.             |

---


**--> à décider ensemble !!!**

In [ ]:
# Liste des colonnes inutiles
hydro_columns_to_drop = [
    'hydro_station_code', 'hydro_status_code', 'hydro_status_label',
    'hydro_method_code', 'hydro_method_label', 'hydro_qualification_code',
    'hydro_qualification_label', 'hydro_longitude', 'hydro_latitude'
]

# Suppression dans les datasets
X_train = X_train.drop(columns=hydro_columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=hydro_columns_to_drop, errors='ignore')

print(f"Colonnes supprimées (Hydrologie) : {len(hydro_columns_to_drop)}")

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.4 Analyse des variables : Prelev

| **Variable**                             | **Description**                                                                                      | **Décision**             | **Justification**                                                                                                                                                            |
|------------------------------------------|------------------------------------------------------------------------------------------------------|---------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `prelev_structure_code_0`                | ID of the water withdrawal facility                                                                  | **Supprimer**            | Non pertinent pour la prédiction des niveaux piezométriques, car ces identifiants ne contribuent pas directement à l’analyse des variables hydrologiques globales.         |
| `prelev_volume_0`                        | Maximum water withdrawal volume (largest annual withdrawal)                                          | **Conserver**            | Variable clé pour estimer l’impact des prélèvements sur les niveaux piezométriques.                                                                                        |
| `prelev_usage_label_0`                   | Goal/usage of the withdrawal                                                                         | **Conserver**            | Permet d’expliquer la répartition des prélèvements selon les usages (eau potable, énergie, irrigation, etc.).                                                              |
| `prelev_volume_obtention_mode_label_0`   | Type of measurement                                                                                 | **Supprimer**            | Peu pertinent pour l’analyse, car la précision du mode d’obtention n’influence pas directement les résultats piezométriques.                                               |
| `prelev_longitude_0`                     | Longitude of the water withdrawal facility                                                          | **Supprimer**            | Déjà supprimée dans les étapes précédentes au profit des coordonnées des stations piezométriques.                                                                           |
| `prelev_latitude_0`                      | Latitude of the water withdrawal facility                                                           | **Supprimer**            | Idem.                                                                                                                                                                       |
| `prelev_commune_code_insee_0`            | City/municipality code where the withdrawal was made                                                | **Supprimer**            | Déjà remplacée par `piezo_station_commune_code_insee` comme référence unique.                                                                                               |
| `prelev_structure_code_1`                | ID of the water withdrawal facility                                                                  | **Supprimer**            | Idem que pour `prelev_structure_code_0`.                                                                                                                                   |
| `prelev_volume_1`                        | Maximum water withdrawal volume (second-largest annual withdrawal)                                  | **Conserver**            | Variable importante pour évaluer l’impact cumulé des prélèvements sur les niveaux piezométriques.                                                                          |
| `prelev_usage_label_1`                   | Goal/usage of the withdrawal                                                                         | **Conserver**            | Idem que pour `prelev_usage_label_0`.                                                                                                                                      |
| `prelev_volume_obtention_mode_label_1`   | Type of measurement                                                                                 | **Supprimer**            | Idem que pour `prelev_volume_obtention_mode_label_0`.                                                                                                                      |
| `prelev_longitude_1`                     | Longitude of the water withdrawal facility                                                          | **Supprimer**            | Idem que pour `prelev_longitude_0`.                                                                                                                                        |
| `prelev_latitude_1`                      | Latitude of the water withdrawal facility                                                           | **Supprimer**            | Idem que pour `prelev_latitude_0`.                                                                                                                                         |
| `prelev_commune_code_insee_1`            | City/municipality code where the withdrawal was made                                                | **Supprimer**            | Idem que pour `prelev_commune_code_insee_0`.                                                                                                                               |
| `prelev_structure_code_2`                | ID of the water withdrawal facility                                                                  | **Supprimer**            | Idem que pour `prelev_structure_code_0`.                                                                                                                                   |
| `prelev_volume_2`                        | Maximum water withdrawal volume (third-largest annual withdrawal)                                   | **Conserver**            | Idem que pour `prelev_volume_0`.                                                                                                                                          |
| `prelev_usage_label_2`                   | Goal/usage of the withdrawal                                                                         | **Conserver**            | Idem que pour `prelev_usage_label_0`.                                                                                                                                      |
| `prelev_volume_obtention_mode_label_2`   | Type of measurement                                                                                 | **Supprimer**            | Idem que pour `prelev_volume_obtention_mode_label_0`.                                                                                                                      |
| `prelev_longitude_2`                     | Longitude of the water withdrawal facility                                                          | **Supprimer**            | Idem que pour `prelev_longitude_0`.                                                                                                                                        |
| `prelev_latitude_2`                      | Latitude of the water withdrawal facility                                                           | **Supprimer**            | Idem que pour `prelev_latitude_0`.                                                                                                                                         |
| `prelev_commune_code_insee_2`            | City/municipality code where the withdrawal was made                                                | **Supprimer**            | Idem que pour `prelev_commune_code_insee_0`.                                                                                                                               |
| `prelev_other_volume_sum`                | Sum of other water withdrawals                                                                      | **Conserver**            | Permet de capturer l’impact global des prélèvements en dehors des trois principaux prélèvements identifiés individuellement.                                               |

---

In [ ]:
# Liste des colonnes inutiles
prelev_columns_to_drop = [
    'prelev_structure_code_0', 'prelev_volume_obtention_mode_label_0', 'prelev_longitude_0', 'prelev_latitude_0', 'prelev_commune_code_insee_0',
    'prelev_structure_code_1', 'prelev_volume_obtention_mode_label_1', 'prelev_longitude_1', 'prelev_latitude_1', 'prelev_commune_code_insee_1',
    'prelev_structure_code_2', 'prelev_volume_obtention_mode_label_2', 'prelev_longitude_2', 'prelev_latitude_2', 'prelev_commune_code_insee_2'
]

# Suppression dans les datasets
X_train = X_train.drop(columns=prelev_columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=prelev_columns_to_drop, errors='ignore')

print(f"Colonnes supprimées (Prélèvement) : {len(prelev_columns_to_drop)}")

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### 2.5 Analyse des variables : INSEE


| **Variable**              | **Description**                                           | **Décision**  | **Justification**                                                                                           |
|---------------------------|-----------------------------------------------------------|---------------|-------------------------------------------------------------------------------------------------------------|
| `insee_%_agri`            | Percentage of the working population in agriculture       | **Conserver** | Donnée pertinente pour évaluer l’impact des activités agricoles sur les prélèvements d’eau et les niveaux piezométriques. |
| `insee_pop_commune`       | Number of inhabitants of the municipality                 | **Conserver** | Variable clé pour estimer la demande en eau, qui peut influencer les prélèvements et les niveaux piezométriques. |
| `insee_med_standard_living` | Median standard of living                                | **Conserver** | Peut refléter indirectement le développement économique et les pressions sur les ressources en eau.         |
| `insee%ind`               | Percentage of the working population in the industry sector | **Conserver** | Reflète l’impact potentiel des activités industrielles sur la consommation et les prélèvements d’eau.       |
| `insee_%_const`           | Percentage of the working population in the construction sector | **Conserver** | Peut refléter les impacts liés aux besoins en eau pour les projets de construction, notamment pour l’irrigation ou le refroidissement. |

---


#### 2.6 Analyse des variables : Distance

| **Variable**             | **Description**                                      | **Décision**  | **Justification**                                                                                     |
|--------------------------|------------------------------------------------------|---------------|-------------------------------------------------------------------------------------------------------|
| `distance_piezo_meteo`   | Distance between the piezometric and weather station | **Conserver** | Permet de mesurer l’influence potentielle des données météorologiques sur les observations piezométriques. |
| `distance_hydro_meteo`   | Distance between the piezometric and hydrological station | **Conserver** | Peut aider à évaluer la cohérence spatiale des données hydrologiques par rapport aux stations piezométriques. |

---



In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
variables_utiles = np.array(X_train.columns)
print(variables_utiles)

## III. Traitement des NaN